In [ ]:
import pandas as pd
import sqlite3
orders = pd.read_csv("/content/orders.csv")
orders.head()



,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [ ]:
users = pd.read_json("/content/users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [ ]:
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()


In [ ]:
with open("/content/restaurants.sql", "r") as file:
    sql_script = file.read()

cursor.executescript(sql_script)
conn.commit()


In [ ]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [ ]:
df = orders.merge(users, on="user_id", how="left")


In [ ]:
df = df.merge(restaurants, on="restaurant_id", how="left")


In [ ]:
df.shape
df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [ ]:
df.to_csv("final_food_delivery_dataset.csv", index=False)


In [ ]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")
df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [ ]:
gold_df = df[df["membership"] == "Gold"]


In [ ]:
gold_city_revenue = (
    gold_df.groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [ ]:
cuisine_aov = (
    df.groupby("cuisine")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

cuisine_aov


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [ ]:
user_total_spend = (
    df.groupby("user_id")["total_amount"]
    .sum()
)


In [ ]:
users_gt_1000 = user_total_spend[user_total_spend > 1000].count()
users_gt_1000


np.int64(2544)

In [ ]:
df["rating_range"] = pd.cut(
    df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0]
)



In [ ]:
rating_revenue = (
    df.groupby("rating_range")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

rating_revenue


/tmp/ipython-input-3226190274.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"]


,total_amount
rating_range,
"(4.5, 5.0]",2197030.75
"(4.0, 4.5]",1960326.26
"(3.0, 3.5]",1881754.57
"(3.5, 4.0]",1717494.41


In [ ]:
import pandas as pd

# Assuming your dataframe is named 'df'
# Filter for Gold members first
gold_members = df[df['membership'] == 'Gold']

# Calculate the average order value (AOV) for each city
city_aov = gold_members.groupby('city')['total_amount'].mean().sort_values(ascending=False)

# Display the result
print("Average Order Value for Gold Members by City:")
print(city_aov)

# To find the highest city specifically:
highest_city = city_aov.idxmax()
highest_value = city_aov.max()

print(f"\nThe city with the highest AOV among Gold members is: {highest_city} (₹{highest_value:.2f})")

Average Order Value for Gold Members by City:
city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

The city with the highest AOV among Gold members is: Chennai (₹808.46)


In [ ]:
cuisine_stats = (
    df.groupby("cuisine")
    .agg(
        distinct_restaurants=("restaurant_id", "nunique"),
        total_revenue=("total_amount", "sum")
    )
    .sort_values("distinct_restaurants")
)

cuisine_stats


,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [ ]:
import pandas as pd

# 1. Count the number of orders for each membership type
membership_counts = df['membership'].value_counts()

# 2. Get the total number of orders in the dataset
total_orders = len(df)

# 3. Extract the number of orders placed by Gold members
gold_order_count = membership_counts.get('Gold', 0)

# 4. Calculate the percentage
gold_percentage = (gold_order_count / total_orders) * 100

print(f"Total Orders: {total_orders}")
print(f"Gold Member Orders: {gold_order_count}")
print(f"Percentage of Gold member orders: {gold_percentage:.2f}%")
print(f"Rounded to nearest integer: {round(gold_percentage)}%")

Total Orders: 10000
Gold Member Orders: 4987
Percentage of Gold member orders: 49.87%
Rounded to nearest integer: 50%


In [ ]:
import pandas as pd

# 1. Group by restaurant name and calculate both the count of orders and the mean (Average Order Value)
restaurant_stats = df.groupby('restaurant_name_y')['total_amount'].agg(['count', 'mean'])

# 2. Filter for restaurants that have fewer than 20 orders
filtered_restaurants = restaurant_stats[restaurant_stats['count'] < 20]

# 3. Sort by the average order value ('mean') in descending order
highest_aov_restaurants = filtered_restaurants.sort_values(by='mean', ascending=False)

# 4. Display the top results
print("Restaurants with < 20 orders, sorted by Average Order Value:")
print(highest_aov_restaurants.head())

# To get the specific name of the top restaurant:
if not highest_aov_restaurants.empty:
    top_restaurant = highest_aov_restaurants.index[0]
    top_value = highest_aov_restaurants.iloc[0]['mean']
    print(f"\nThe restaurant with the highest AOV and < 20 orders is: {top_restaurant} (₹{top_value:.2f})")

Restaurants with < 20 orders, sorted by Average Order Value:
                   count         mean
restaurant_name_y                    
Restaurant_294        13  1040.222308
Restaurant_262        18  1029.473333
Restaurant_77         12  1029.180833
Restaurant_193        15  1026.306667
Restaurant_7          16  1002.140625

The restaurant with the highest AOV and < 20 orders is: Restaurant_294 (₹1040.22)


In [ ]:
combo_revenue = (
    df.groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .reset_index()
)


In [ ]:
mcq_combinations = combo_revenue[
    (
        (combo_revenue["membership"] == "Gold") &
        (combo_revenue["cuisine"].isin(["Indian", "Italian"]))
    ) |
    (
        (combo_revenue["membership"] == "Regular") &
        (combo_revenue["cuisine"].isin(["Indian", "Chinese"]))
    )
]

mcq_combinations.sort_values("total_amount", ascending=False)


,membership,cuisine,total_amount
2,Gold,Italian,1005779.05
5,Regular,Indian,992100.27
1,Gold,Indian,979312.31
4,Regular,Chinese,952790.91


In [27]:
df["order_date"] = pd.to_datetime(df["order_date"])


/tmp/ipython-input-760424543.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


In [28]:
df["order_date"] = pd.to_datetime(df["order_date"])


In [30]:
df["quarter"] = df["order_date"].dt.quarter
quarter_revenue = (
    df.groupby("quarter")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

quarter_revenue

,total_amount
quarter,
3,2037385.10
4,2018263.66
1,2010626.64
2,1945348.72


In [31]:
import pandas as pd

# Load dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

print("===== DATA VERIFICATION RESULTS =====\n")

# 1. Total orders by Gold members
gold_total_orders = df[df["membership"] == "Gold"].shape[0]
print("1️⃣ Total orders by Gold members:", gold_total_orders)

# 2. Total revenue from Hyderabad (rounded)
hyderabad_revenue = round(
    df[df["city"] == "Hyderabad"]["total_amount"].sum()
)
print("2️⃣ Total revenue from Hyderabad (rounded):", hyderabad_revenue)

# 3. Distinct users who placed at least one order
distinct_users = df["user_id"].nunique()
print("3️⃣ Distinct users who placed at least one order:", distinct_users)

# 4. Average order value for Gold members (rounded to 2 decimals)
gold_avg_order_value = round(
    df[df["membership"] == "Gold"]["total_amount"].mean(), 2
)
print("4️⃣ Average order value for Gold members:", gold_avg_order_value)

# 5. Orders placed for restaurants with rating >= 4.5
orders_rating_ge_45 = df[df["rating"] >= 4.5].shape[0]
print("5️⃣ Orders with restaurant rating ≥ 4.5:", orders_rating_ge_45)

# 6. Orders placed in the top revenue city among Gold members
top_gold_city = (
    df[df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

orders_top_gold_city = (
    df[(df["membership"] == "Gold") & (df["city"] == top_gold_city)]
    .shape[0]
)

print("6️⃣ Top revenue city among Gold members:", top_gold_city)
print("   Orders in that city (Gold only):", orders_top_gold_city)

print("\n===== END OF VERIFICATION =====")


===== DATA VERIFICATION RESULTS =====

1️⃣ Total orders by Gold members: 4987
2️⃣ Total revenue from Hyderabad (rounded): 1889367
3️⃣ Distinct users who placed at least one order: 2883
4️⃣ Average order value for Gold members: 797.15
5️⃣ Orders with restaurant rating ≥ 4.5: 3374
6️⃣ Top revenue city among Gold members: Chennai
   Orders in that city (Gold only): 1337

===== END OF VERIFICATION =====


In [32]:
import pandas as pd

orders = pd.read_csv("orders.csv")
users = pd.read_json("users.json")

print("Orders columns:", orders.columns.tolist())
print("Users columns:", users.columns.tolist())


Orders columns: ['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name']
Users columns: ['user_id', 'name', 'city', 'membership']


In [33]:
# Just checking file extension
import os

os.listdir()


['.config',
 'final_food_delivery_dataset.csv',
 'orders.csv',
 'restaurants.db',
 'users.json',
 'restaurants.sql',
 'sample_data']

In [34]:
final_df = pd.read_csv("final_food_delivery_dataset.csv")
orders = pd.read_csv("orders.csv")

print("Rows in orders.csv:", orders.shape[0])
print("Rows in final dataset:", final_df.shape[0])


Rows in orders.csv: 10000
Rows in final dataset: 10000


In [35]:
merged = orders.merge(users, on="user_id", how="left")

merged.isna().sum()


,0
order_id,0
user_id,0
restaurant_id,0
order_date,0
total_amount,0
restaurant_name,0
name,0
city,0
membership,0


In [36]:
# Example merge command used
orders.merge(users, on="user_id", how="left")


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold


In [37]:
final_df["user_id"].value_counts().head()


,count
user_id,
2973,13
1515,12
874,11
496,11
1337,11
